# Variational Autoencoders in edward2

A similar tutorial is written with tfp:
https://github.com/tensorflow/probability/blob/master/tensorflow_probability/examples/jupyter_notebooks/Bayesian_Gaussian_Mixture_Model.ipynb

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="7"

In [2]:
from functools import partial
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
Variable = tf.Variable
import tensorflow_probability as tfp
tfd = tfp.distributions
ed = tfp.edward2

print('TensorFlow version {}, TF Probability version {}.'.format(tf.__version__, tfp.__version__))

TensorFlow version 1.10.0, TF Probability version 0.4.0.
